In [ ]:
# First reset the pygkyl library
import sys
!rm -rf ~/personal_gkyl_scripts/pygkyl/pygkyl.egg-info
!rm -rf ~/personal_gkyl_scripts/pygkyl/build
!{sys.executable} -m pip install ~/personal_gkyl_scripts/pygkyl > ~/personal_gkyl_scripts/pygkyl/install.log

import numpy as np
import matplotlib.pyplot as plt
import os

# Configure plotting
plt.rcParams["figure.figsize"] = (6,4)

# Custom libraries and routines
import pygkyl

home_dir = os.path.expanduser("~")
repo_dir = home_dir+'/personal_gkyl_scripts/'
# simdir = repo_dir+'sim_data_dir_example/par_adv_test/'
# fileprefix = 'rt_gk_tcv_nt_iwl_3x2v_p1'
simdir = 'sim_data_dir_example/3x2v_example/gk_tcv_posD_iwl_3x2v_electron_heating/'
fileprefix = 'gk_tcv_posD_iwl_3x2v_D02'

simulation = pygkyl.simulation_configs.import_config('tcv_nt', simdir, fileprefix)
# simulation = pygkyl.simulation_configs.import_config('d3d_nt', simdir, fileprefix)
# simulation.geom_param.x_LCFS = 0.08

simulation.normalization.set('t','mus') # time in micro-seconds
simulation.normalization.set('x','minor radius') # radial coordinate normalized by the minor radius (rho=r/a)
simulation.normalization.set('y','Larmor radius') # binormal in term of reference sound Larmor radius
simulation.normalization.set('z','pi') # parallel angle devided by pi
simulation.normalization.set('fluid velocities','thermal velocity') # fluid velocity moments are normalized by the thermal velocity
simulation.normalization.set('temperatures','eV') # temperatures in electron Volt
simulation.normalization.set('pressures','Pa') # pressures in Pascal
simulation.normalization.set('energies','MJ') # energies in mega Joules

sim_frames = simulation.available_frames['ion_BiMaxwellianMoments'] # you can check the available frames for each data type like ion_M0, ion_BiMaxwellian, etc.)
print("%g time frames available (%g to %g)"%(len(sim_frames),sim_frames[0],sim_frames[-1]))

In [ ]:
# !{sys.executable} -m pip install trame-vtk pyvista trame-vuetify imageio

In [ ]:
fieldname = 'ni'
rho = 1.2
Nint = 32
overSampFact = 2
timeFrame = sim_frames[-10]
# pygkyl.plot_utils.flux_surface_proj(simulation, fieldName=fieldname, rho=rho, timeFrame=timeFrame, Nint=Nint, overSampFact=overSampFact)
fsproj = pygkyl.FluxSurfProjection()
fsproj.setup(simulation, Nint=Nint, rho=rho, smooth=True)
fsproj.plot(fieldName=fieldname, timeFrame=timeFrame, fluctuation='')

In [ ]:
fsproj.plot(fieldName=fieldname, timeFrame=timeFrame, fluctuation='yavg_relative')

In [ ]:
Nint_polproj = 24
Nint_fsproj = 32
torproj = pygkyl.TorusProjection()
torproj.setup(simulation, Nint_polproj=Nint_polproj, Nint_fsproj=Nint_fsproj, 
              phiLim = [0, 3*np.pi/2], rhoLim = [0.86,1.3])

In [ ]:
torproj.plot(fieldName='ni', timeFrame=sim_frames[-1], fluctuation='',clim=[17,19.3], colorScale='log', 
             vessel=True, outFilename='torproj_ni.png', imgSize=(800,600), lighting=False, smooth_shading=False)

In [ ]:
torproj.movie(fieldName='ni', timeFrames=sim_frames[-100:], fluctuation='',clim=[17,19.3], colorScale='log', 
             vessel=True, filePrefix='d3d_geom_', imgSize=(800,600), smooth_shading=False, lighting=False)

In [ ]:
adsfg

In [ ]:
polproj = pygkyl.PoloidalProjection()
polproj.setup(simulation,nzInterp=16) # we use rholim but we can also use Rlim
polproj.toroidal_rotate(+8*np.pi/4)
fieldname = 'ni'
rho = 0.9
Nint = 16 # better results with Nint>=128
overSampFact = 2 # better results with overSampFact>=2
timeFrame = sim_frames[-1]
pngname = 'ni_full_torus.png'
polproj.plot_full_torus(fieldname, timeFrame, rho=rho, Nint=Nint, overSampFact=overSampFact,
                        colorScale='linear', elev=15, azim=-60, clim=[], philim=[0,5*np.pi/4],
                        outFilename=pngname, dpi=150, fluctuation='yavg')
# display the generated png
from IPython.display import Image
Image(filename=pngname)

In [ ]:
sdfgs

In [ ]:
def qprofile(rIn):
    qa = [497.3420166252413,-1408.736172826569,1331.4134861681464,-419.00692601227627]
    # qa = [0, 0, 1.5, 0.0]
    R = rIn + simulation.geom_param.R_axis
    return (qa[0]*R**3 + qa[1]*R**2 + qa[2]*R + qa[3])
    # return 2
simulation.geom_param.set_qprofile(qprofile)

polproj = pygkyl.PoloidalProjection()
polproj.setup(simulation,nzInterp=16, gridCheck=False, TSBC=True, zExt=True)

In [ ]:
fieldname = 'ni'
xcut = 0.95
Nint = 128
overSampFact = 1
pygkyl.plot_utils.flux_surface_proj(simulation, fieldName=fieldname, xCut=xcut, timeFrame=0, Nint=Nint, overSampFact=overSampFact)

In [ ]:
# polproj.inset.xlim = [0.65,0.75]
# polproj.set_toroidal_rotation(0*np.pi)
# polproj.plot('ni',timeFrame=sim_frames[0],colorScale='lin',fluctuation='',clim=[],colorMap='',inset=False)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class Context:
    a_shift = 0.5  # Parameter in Shafranov shift
    Z_axis = 0.1414361745  # Magnetic axis height [m]
    R_axis = 0.8867856264  # Magnetic axis major radius [m]
    B_axis = 1.4  # Magnetic field at the magnetic axis [T]
    R_LCFSmid = 1.0870056099999  # Major radius of the LCFS at the outboard midplane [m]
    x_inner = 0.04  # Radial extent inside LCFS
    x_outer = 0.08  # Radial extent outside LCFS
    Rmid_min = R_LCFSmid - x_inner  # Minimum midplane major radius of simulation box [m]
    Rmid_max = R_LCFSmid + x_outer  # Maximum midplane major radius of simulation box [m]
    R0 = 0.5 * (Rmid_min + Rmid_max)  # Major radius of the simulation box [m]
    a_mid = R_LCFSmid - R_axis  # Minor radius at outboard midplane [m]
    # Redefine a_mid with Shafranov shift, to ensure LCFS radial location
    a_mid = R_axis / a_shift - np.sqrt(R_axis * (R_axis - 2 * a_shift * R_LCFSmid + 2 * a_shift * R_axis)) / a_shift
    r0 = R0 - R_axis  # Minor radius of the simulation box [m]
    Lx = Rmid_max - Rmid_min
    x_min = 0.0
    x_max = Lx
    Ly = 0.2
    Lz = 2.0 * np.pi - 1e-10
    Nx = 48
    
    def r_x(self, x):
        return x + self.a_mid - self.x_inner
    
ctx = Context()

def qprofile_TCV(r, ctx):
    y = r + ctx.R_axis
    a = 484.0615913225881
    b = -1378.25993228584
    c = 1309.3099150729233
    d = -414.13270311478726
    return (a*y**3 + b*y**2 + c*y + d)
    return 0.5+2*y


def shear(r, qprof, ctx):
    q = qprof(r, ctx)  # Assuming q is callable
    dqdr = np.gradient(q, r)
    shear = (r/q) * dqdr
    return shear

def shift(x, qprofile, ctx):
    x0 = 0.5 * (ctx.x_min + ctx.x_max)
    r0 = ctx.r_x(x0)
    q0 = qprofile(r0, ctx)  # Assuming qprofile is callable
    r = ctx.r_x(x)
    return -ctx.r0 / q0 * qprofile(r,ctx) * ctx.Lz


x = np.linspace(ctx.x_min, ctx.x_max, ctx.Nx) # radial coordinate
r = ctx.r_x(x) # minor radius
R = ctx.R_axis + r # major radius

# linear fit of the TCV q-profile
qprofile = qprofile_TCV(r, ctx)
qfit_lin = np.polyfit(R, qprofile, 1)
def qfit_lin_func(r, ctx):
    y = r + ctx.R_axis
    return 1.5 * y 
    return qfit_lin[0] * y + qfit_lin[1]
# print the coefficients of the linear fit
print("Linear fit coefficients: ", qfit_lin)

# quadratic fit of the TCV q-profile
qfit_quad = np.polyfit(R, qprofile, 2)
def qfit_quad_func(r, ctx):
    y = r + ctx.R_axis
    return qfit_quad[0] * y**2 + qfit_quad[1] * y + qfit_quad[2]
# print the coefficients of the quadratic fit
print("Quadratic fit coefficients: ", qfit_quad)

# cubic fit of the TCV q-profile
qfit_cub = np.polyfit(R, qprofile, 3)
def qfit_cub_func(r, ctx):
    y = r + ctx.R_axis
    return qfit_cub[0] * y**3 + qfit_cub[1] * y**2 + qfit_cub[2] * y + qfit_cub[3]

# piecewise linear fit of the TCV q-profile
Npieces = 8
qfit_piecewise = np.zeros((Npieces, 2))
for i in range(Npieces):
    x1 = i * (ctx.x_max - ctx.x_min) / Npieces
    x2 = (i + 1) * (ctx.x_max - ctx.x_min) / Npieces
    r1 = ctx.r_x(x1)
    r2 = ctx.r_x(x2)
    R1 = r1 + ctx.R_axis
    R2 = r2 + ctx.R_axis
    q1 = qprofile_TCV(r1, ctx)
    q2 = qprofile_TCV(r2, ctx)
    qfit_piecewise[i, 0] = (q2 - q1) / (R2 - R1)  # slope
    qfit_piecewise[i, 1] = q1 - qfit_piecewise[i, 0] * R1  # intercept

    
def qfit_piecewise_func(r, ctx):
    if isinstance(r, float): 
        r = np.array([r])
    y = r + ctx.R_axis
    fit = np.zeros_like(y)
    for i in range(len(y)):
        for j in range(Npieces):
            x1 = j * (ctx.x_max - ctx.x_min) / Npieces
            x2 = (j + 1) * (ctx.x_max - ctx.x_min) / Npieces
            R1 = ctx.r_x(x1) + ctx.R_axis
            R2 = ctx.r_x(x2) + ctx.R_axis
            if y[i] >= R1 and y[i] <= R2:
                fit[i] = qfit_piecewise[j, 0] * y[i] + qfit_piecewise[j, 1]
    return fit
    
# print the coefficients of the piecewise linear fit and their interval in a c code format with 16 digits
print("Piecewise linear fit coefficients in C code format: ")
for i in range(Npieces):
    x1 = i * (ctx.x_max - ctx.x_min) / Npieces
    x2 = (i + 1) * (ctx.x_max - ctx.x_min) / Npieces
    r1 = ctx.r_x(x1)
    r2 = ctx.r_x(x2)
    R1 = r1 + ctx.R_axis
    R2 = r2 + ctx.R_axis
    print("if (R >= %.14g && R <= %.14g) q = %.14g * R + %.14g;" % (R1, R2, qfit_piecewise[i, 0], qfit_piecewise[i, 1]))


# compare the fits and the original TCV data for q profile, shear and shift
fig, ax = plt.subplots(1, 3, figsize=(12, 5))
# Plot q-profile
ax[0].plot(R, qprofile, '-k', label='Original TCV')
ax[0].plot(R, qfit_lin_func(r, ctx), '-', label='Linear Fit')
# ax[0].plot(R, qfit_quad_func(r, ctx), '-', label='Quadratic Fit')
# ax[0].plot(R, qfit_cub_func(r, ctx), '-', label='Cubic Fit')
ax[0].plot(R, qfit_piecewise_func(r, ctx), '--c', label='Piecewise Linear Fit')
ax[0].set_xlabel('Major Radius [m]')
ax[0].set_ylabel('q-profile')
ax[0].set_title('q-profile')
# Plot shear
shear_original = shear(r, qprofile_TCV, ctx)
shear_linear = shear(r, qfit_lin_func, ctx)
shear_quadratic = shear(r, qfit_quad_func, ctx)
ax[1].plot(R, shear_original, '-k')
ax[1].plot(R, shear_linear, '-')
# ax[1].plot(R, shear_quadratic, '-')
# ax[1].plot(R, shear(r, qfit_cub_func, ctx), '-')
ax[1].plot(R, shear(r, qfit_piecewise_func, ctx), '--c')
ax[1].set_xlabel('Major Radius [m]')
ax[1].set_ylabel(r'$r/q(r) \times \partial q(r)/\partial r$')
ax[1].set_title('Shear')
# Plot shift
shift_original = shift(x, qprofile_TCV, ctx)/ctx.Ly
shift_linear = shift(x, qfit_lin_func, ctx)/ctx.Ly
shift_quadratic = shift(x, qfit_quad_func, ctx)/ctx.Ly
ax[2].plot(R, shift_original, '-k')
ax[2].plot(R, shift_linear, '-')
# ax[2].plot(R, shift_quadratic, '-')
# ax[2].plot(R, shift(x, qfit_cub_func, ctx)/ctx.Ly, '-')
ax[2].plot(R, shift(x, qfit_piecewise_func, ctx)/ctx.Ly, '--c')
ax[2].set_xlabel('Major Radius [m]')
ax[2].set_ylabel(r'$S_y/L_y$')
ax[2].set_title('Shift')
# Add legend only to the first subplot
ax[0].legend()#loc='upper center', bbox_to_anchor=(1.5, 1.2), ncol=3)
plt.tight_layout()
plt.show()


In [ ]:

# Create a 2x1 figure for combined plots
fig, axs = plt.subplots(1, 3, figsize=(12, 4))

# Plot the q-profile
axs[0].set_xlabel('R (m)')
axs[0].set_ylabel('q-profile')
axs[0].plot(R, qprofile_TCV(r,ctx), '-k', label='TCV')
axs[0].plot(R, qprofile_lowshear(r,ctx), '-b', label='low shear ')
axs[0].plot(R, qprofile_highshear(r,ctx), '-r', label='high shear')
eta = 0.675
axs[0].plot(R, qprofile_intermediate(r,ctx), '-g', label='mix (eta=%.2f)'%eta)
eta = 0.676
axs[0].plot(R, qprofile_intermediate(r,ctx), '--y', label='mix (eta=%.2f)'%eta)
axs[0].grid()

# Plot the shear
axs[1].set_xlabel('R (m)')
axs[1].set_ylabel(r'Shear $r/q \partial_r q$')
axs[1].plot(R, shear(r, qprofile_TCV, ctx), '-k', label='TCV')
axs[1].plot(R, shear(r, qprofile_lowshear, ctx), '-b', label='low shear ')
axs[1].plot(R, shear(r, qprofile_highshear, ctx), '-r', label='high shear')
eta = 0.675
axs[1].plot(R, shear(r, qprofile_intermediate, ctx), '-g', label='mix (eta=%.2f)'%eta)
eta = 0.676
axs[1].plot(R, shear(r, qprofile_intermediate, ctx), '--y', label='mix (eta=%.2f)'%eta)
axs[1].grid()

# Plot the shift function
axs[2].set_xlabel('R (m)')
axs[2].set_ylabel(r'Shift function $S(x)/L_y$')
axs[2].plot(R, shift(x, qprofile_TCV, ctx)/ctx.Ly, '-k', label='TCV')
axs[2].plot(R, shift(x, qprofile_lowshear, ctx)/ctx.Ly, '-b', label='low shear ')
axs[2].plot(R, shift(x, qprofile_highshear, ctx)/ctx.Ly, '-r', label='high shear')
eta = 0.675
axs[2].plot(R, shift(x, qprofile_intermediate, ctx)/ctx.Ly, '-g', label='mix (eta=%.3f)'%eta)
eta = 0.676
axs[2].plot(R, shift(x, qprofile_intermediate, ctx)/ctx.Ly, '--y', label='mix (eta=%.3f)'%eta)
axs[2].grid()

# Add a single legend outside the last plot
handles, labels = axs[2].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=6)

# Adjust layout and show the figure
plt.tight_layout()
plt.show()

In [ ]:
etas = np.linspace(0.6, 0.7, 100)
shifts = [np.min(shift(x, lambda r, ctx: (1 - eta) * qprofile_lowshear(r, ctx) + eta * qprofile_highshear(r, ctx), ctx) / ctx.Ly) for eta in etas]
plt.plot(etas, np.abs(shifts), '-')
# plot vertical line at eta=0.675
plt.axvline(x=0.675, color='k', linestyle='--')
# plot horizontal line at the shift value when eta=0.675
idx = np.argmin(np.abs(etas - 0.675))
plt.axhline(y=np.abs(shifts[idx]), color='k', linestyle='--')
plt.xlabel('eta')
plt.ylabel('Minimum shift')

In [ ]:
eta = [  0.5,   0.65,  0.675, 0.676,   0.677,   0.68,    0.7,  0.71,  0.72,  0.725,   0.73,   0.74,  0.75,    0.76,   0.8,   0.85,   0.9,  0.95,   1.0]
err = [1e-13,  1e-13,  1e-13, 0.0012, 0.0034, 0.0058, 0.0058, 0.006, 0.0062, 0.0015, 0.0063, 0.006, 0.0032, 0.0042, 0.005, 0.0077, 0.0145, 0.023, 0.035]
fig, ax = plt.subplots()
ax.semilogy(eta, [100*e for e in err], '.--')
ax.set_xlabel(r'$\eta$')
ax.set_ylabel(r'error [\%]')
ax.set_title('Error of the twist-and-shift')
ax.grid()
ax.set_ylim(0, 4)
ax.set_xlim(0.5, 1.0)
plt.show()

In [ ]:
import numpy as np
kappa = 1.3
delta = -0.2
q0 = 1.4
qa = 2.5
a = 0.9
R0 = 1.2

def q(r):
    return q0 + (qa - q0) * (r/a)**2

def s(r):
    return 2/a * (qa - q0) * (r/a)

Sma = [0, 0, 0,-a*(kappa-1)/(kappa+1), a*delta/4]

def S(m,r):
    return Sma[m] * np.power(r/a,m-1) * (q(r)*s(r) + 2*q0*(m+1)/(m-1))/(qa*s(a) + 2*q0*(m+1)/(m-1))

def Rc(r,theta):
    sum1 = 0
    sum2 = 0
    for m in [2,3]:
        sum1 += S(m,r)/R0 * np.cos((m-1)*theta)
        sum2 += (1-m)/(2*r*R0)*(S(m,r))**2 * np.cos(theta)
        
    return R0 * (1 + r/R0*np.cos(theta) + sum1 - sum2)
    
def Zc(r,theta):
    sum1 = 0
    sum2 = 0
    for m in [2,3]:
        sum1 += S(m,r)/R0 * np.sin((m-1)*theta)
        sum2 += (1-m)/(2*r*R0)*(S(m,r))**2 * np.sin(theta)
        
    return R0*(r/R0*np.sin(theta) - sum1 - sum2)

def RcTB(r,theta):
    return R0 \
            + r*np.cos(theta) \
            + r*(kappa-1)/(kappa+1)*(q(r)*s(a) + 6*q0)/(qa*s(a) + 6*q0)*np.cos(theta) \
            + delta*r**2/(4*a)*(q(r)*s(a) + 4*q0)/(qa*s(a) + 4*q0)*np.cos(2*theta)
            
def ZcTB(r,theta):
    return r*np.sin(theta) \
        - r*(kappa-1)/(kappa+1)*(q(r)*s(a) + 6*q0)/(qa*s(a) + 6*q0)*np.sin(theta) \
        - delta*r**2/(4*a)*(q(r)*s(a) + 4*q0)/(qa*s(a) + 4*q0)*np.sin(2*theta)
        
r = np.linspace(0.1, a, 100)
theta = np.linspace(0, 2*np.pi, 100)

R_vals = np.zeros((len(r), len(theta)))
Z_vals = np.zeros((len(r), len(theta)))
for i in range(len(r)):
    for j in range(len(theta)):
        R_vals[i,j] = Rc(r[i], theta[j])
        Z_vals[i,j] = Zc(r[i], theta[j])
        
# Tess version
R_valsTB = np.zeros((len(r), len(theta)))
Z_valsTB = np.zeros((len(r), len(theta)))
for i in range(len(r)):
    for j in range(len(theta)):
        R_valsTB[i,j] = RcTB(r[i], theta[j])
        Z_valsTB[i,j] = ZcTB(r[i], theta[j])
# Plot the results
plt.figure()
for n in range(len(r))[::10]:
    plt.plot(R_vals[n,:], Z_vals[n,:],'b-')
    plt.plot(R_valsTB[n,:], Z_valsTB[n,:],'r-')
    
plt.xlabel('R (m)')
plt.ylabel('Z (m)')
plt.title('Magnetic geometry')
plt.axis('equal')
plt.show()

plt.figure()
# plot q profile and shear
plt.subplot(2,1,1)
plt.plot(r, q(r), 'b-', label='q-profile')
plt.plot(r, s(r), 'r-', label='shear')

In [ ]:
## FIX KY FOURIER

cut_dir = 'xky' # the plane we want to plot
cut_coord = 0.0 # the coordinate were the plan stands If normalized units are defined, 
# this number is in the normalize units. One can also specify a integer for array index 
# or 'avg' to get an average over the reduced dim.
time_frame = sim_frames[-1] # the time frame
fieldnames = ['ne'] # the fields to plot, see simulation.display_available_fields() (some may not work in 2x2v)
pygkyl.plot_utils.plot_2D_cut(simulation, cut_dir, cut_coord, time_frame,fieldnames,plot_type='pcolormesh')